In [1]:
%pylab
%load_ext line_profiler
from numba import jit, njit

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [3]:
class NTree:
    """Arbitrary dimensionality n-ant tree structure that stores arbitrary data associated with each point."""
    def __init__(self, center, size, dim=3):
        self.COM = None
        self.center = center
        self.size = size
        self.data = None
        self.IsLeaf = False
        self.children = (1 << dim) * [None,]
        self.dim = dim
        
    def InsertPoint(self, x, data=None):
        """Inserts a point of position x and mass m into the tree."""
        if self.COM is None: # no point already lives here, so let's make a leaf node and store the point there
            self.COM  = x
            if data: self.data = data
            self.IsLeaf = True
            return
        #otherwise we gotta split this up
        if self.IsLeaf:
            self.SpawnChildWithPoint(self.COM, data)
            self.IsLeaf = False    
        self.SpawnChildWithPoint(x, data)
    
    def SpawnChildWithPoint(self, x, data=None):
        """Spawns a child node for a point at position x and mass m to live in."""
        signs = (x > self.center)
        #sector, signs = Sector(x, self.center)
        sector = SignsToSector(signs) # number from 0 to 2**dim - 1 deciding which n-ant 
        #print(sector is sector2)
        if not self.children[sector]:
            child_size = self.size/2
            child_center =  self.center + child_size*(signs-0.5)
            self.children[sector] = NTree(child_center, child_size, dim=self.dim)
        self.children[sector].InsertPoint(x, data)
        
    def GetMoments(self):
        """Computes the mass and center of mass of a node recursively."""
        if not self.IsLeaf: #: return self.mass, self.COM
            self.data = 0.
            self.COM = np.zeros(self.dim)
            for c in self.children:
                if c is None: continue
                mc, xc = c.GetMoments()
                self.data += mc
                self.COM += mc*xc
            self.COM /= self.data
        return self.data, self.COM

@njit
def SignsToSector(signs):
    """Takes a boolean array and returns the integer given by those binary digits."""
    sum = 0
    for i in range(signs.shape[0]):
        #sum += signs[i] * (1 << i)
        if signs[i]: sum += 1 << i
    return sum

@njit 
def Sector(x, center):
    """Returns a number from 0 to 2**dim - 1 labeling which n-ant the point lives in"""
    sum = 0
    #signs = np.zeros(3, dtype=np.bool)
    for i in range(center.shape[0]):
        if x[i] > center[i]: 
            sum += 1 << i
            #signs[i] = True
    return sum
    
def ForceWalk(x, g, node, thetamax=0.7, eps=0.0):
    dx = node.COM - x
    #print(dx)
    r = np.sqrt((dx**2).sum())
    if r>0:
        if node.IsLeaf or node.size/r < thetamax:
            g += node.mass * dx / (r**2 + eps**2)**1.5
        else:
            for c in node.children:
                if c: ForceWalk(x, g, c, thetamax, eps)

def Accel(points, tree, thetamax=0.7, G=1.0, eps=0.0):
    accels = np.zeros_like(points)
    for i in range(points.shape[0]):
        ForceWalk(points[i], accels[i], tree, thetamax,eps)
    return G*accels

@njit
def BruteForceAccel(x,m,eps=0., G=1.):
    accel = zeros_like(x)
    for i in range(x.shape[0]):
        for j in range(i+1,x.shape[0]):
            dx = x[j,0]-x[i,0]
            dy = x[j,1]-x[i,1]
            dz = x[j,2]-x[i,2]
            r = sqrt(dx*dx + dy*dy + dz*dz + eps*eps)
            mr3inv = m[i]/(r*r*r)
            accel[j,0] -= mr3inv*dx
            accel[j,1] -= mr3inv*dy
            accel[j,2] -= mr3inv*dz

            mr3inv = m[j]/(r*r*r)
            accel[i,0] += mr3inv*dx
            accel[i,1] += mr3inv*dy
            accel[i,2] += mr3inv*dz
    return G*accel

@jit
def BruteForcePotential(x,m,G=1., eps=0.):
    potential = np.zeros_like(m)
    for i in range(x.shape[0]):
        for j in range(i+1,x.shape[0]):
            dx = x[i,0]-x[j,0]
            dy = x[i,1]-x[j,1]
            dz = x[i,2]-x[j,2]
            r = np.sqrt(dx*dx + dy*dy + dz*dz + eps*eps)
            rinv = 1/r
            potential[j] -= m[i]*rinv
            potential[i] -= m[j]*rinv
    return G*potential

def ConstructTree(points, data=None):
    mins = np.min(points,axis=0)
    maxes = np.max(points,axis=0)
    center = (maxes+mins)/2
    size = np.max(maxes-mins)
    root = NTree(center, size, dim=points.shape[1])
    if data:
        for i in range(len(points)):
            root.InsertPoint(points[i], data[i])#, masses[i])
    else:
        for i in range(len(points)):
            root.InsertPoint(points[i])
    #root.GetMoments()
    return root


In [4]:
x = 2*(np.random.rand(10**5,3) - 0.5)
#x = x[np.sum(x**2,axis=1)<1.]
#x[:,2] /= 10
masses = np.repeat(1/x.shape[0],x.shape[0])
#v = np.cross(x, np.array([0,0,1])) * 3
#v += np.random.normal(size=x.shape)*0.1
#v *= 0.
#plt.scatter(x[:,0], x[:,1]); plt.show()

In [144]:
%lprun -f TreeNode.InsertPoint  ConstructTree(x, masses)

In [5]:
%time ConstructTree(x)

CPU times: user 1.84 s, sys: 45.3 ms, total: 1.88 s
Wall time: 1.89 s


In [133]:
%timeit ConstructTree(x, masses)

23.8 s ± 321 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [48]:
#g = np.zeros(3)
%time root = ConstructTree(x, masses)
#a = Accel(x, root, thetamax=0.7,eps=0.1)
#a[0], BruteForceAccel(x,masses,eps)[0]
#plt.hist(a[:,1],100); plt.show()
#root.children#[2].center
#x[np.sum(a**2,axis=1).argmax()]
#BruteForceAccel(points, masses)

CPU times: user 25.8 s, sys: 267 ms, total: 26.1 s
Wall time: 26 s


In [15]:
dt = 0.001
eps = 0.1
t = 0.
tmax = 1.
i = 0
#plt.ion()
#ion()
#fig = plt.figure()
#ax = fig.add_subplot(111)
#plt.axes().set_aspect('equal'); 
#plt.xlim(-1,1)
#plt.ylim(-1,1)
KE = []
PE = []
med = []
while t < tmax:
    if not i%100: print(t)# ax.clear(); ax.scatter(x[:,0],x[:,1],s=0.3); plt.xlim(-1,1); plt.ylim(-1,1); plt.draw(); plt.pause(0.01)
    #plt.savefig("%d.png"%i); plt.plt.clf()
    x += v*dt #, v + BruteForceAccel(x, masses, eps=eps)*dt
    #root = ConstructTree(x, masses)
    v += BruteForceAccel(x, masses, eps=eps)*dt
    i += 1
    t += dt
    KE.append((v**2).sum())
    PE.append(BruteForcePotential(x,masses,1.,eps).sum())
    med.append(np.percentile(np.sum(x**2,axis=1)**0.5, 50))

0.0
0.10000000000000007
0.20000000000000015
0.3000000000000002
0.4000000000000003
0.5000000000000003
0.6000000000000004
0.7000000000000005
0.8000000000000006
0.9000000000000007


In [17]:
plt.plot(np.array(KE) + np.array(PE))
#plt.plot(PE)
plt.show()

In [86]:
plt.plot(med); plt.show()

In [ ]:
np.packbits(np.array([True,True,True]))

In [43]:
signs = np.random.rand(3) > 0.5
#%timeit sum(signs[i] * (1 << i) for i in range(3))
%timeit SignsToIndex(signs)

227 ns ± 0.146 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [136]:
%timeit center + 0.5*(signs-0.5)

1.4 µs ± 29.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [32]:
from numba import uint8, boolean

In [42]:
@jit
def SignsToIndex(signs):
    sum = 0
    for i in range(signs.shape[0]):
        sum += signs[i] * (1 << i)
    return sum

@jit 

In [151]:
x = np.random.rand(3)*2 - 1
center = np.array([0.1,0.1,0.1])



In [157]:
signs = (x > center)
%timeit SignsToSector(signs) # number from 0 to 2**dim deciding which n-ant 

223 ns ± 0.258 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
